In [3]:
import os
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

def ensure_directories_exist():
    """Ensure all required directories exist."""
    os.makedirs('Plots', exist_ok=True)
    os.makedirs('Data', exist_ok=True)
    os.makedirs('Networks', exist_ok=True)

def initialize_infection(N, ratio):
    """Initialize the infected population based on the given ratio."""
    infected = np.zeros(N, dtype=bool)
    infected[:int(N * ratio)] = True
    np.random.shuffle(infected)
    return infected

def sis_simulation(G, beta, mu, initial_ratio, Tmax, Ttrans):
    """Perform SIS model simulation."""
    N = G.number_of_nodes()
    infected = initialize_infection(N, initial_ratio)
    results = []

    for _ in range(Tmax):
        new_infected = infected.copy()
        for node in G.nodes:
            if infected[node] and np.random.random() < mu:
                new_infected[node] = False
            elif not infected[node]:
                if any(np.random.random() < beta for neighbor in G.neighbors(node) if infected[neighbor]):
                    new_infected[node] = True
        infected = new_infected
        if _ >= Ttrans:
            results.append(np.mean(infected))
    return np.mean(results)

def mmca_prediction(G, beta, mu):
    """Predict the stationary state using the MMCA method."""
    N = G.number_of_nodes()
    A = nx.to_numpy_array(G)
    p = np.random.rand(N)  # Initial probabilities of being infected
    for _ in range(1000):
        new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
        if np.allclose(p, new_p, atol=1e-4):
            break
        p = new_p
    return np.mean(p)

def create_networks(N):
    """Generate various network types with specified parameters."""
    return {
        "Erdos_Renyi": nx.erdos_renyi_graph(n=N, p=0.01),
        "Scale_Free": nx.barabasi_albert_graph(n=N, m=5),
        "Small_World": nx.watts_strogatz_graph(n=N, k=20, p=0.1)
    }

def run_simulations():
    """Setup and run simulations across different network configurations."""
    ensure_directories_exist()
    network_sizes = [500, 1000, 1500]
    beta_values = np.linspace(0, 1, 51)
    mu_values = [0.1, 0.3, 0.5, 0.7, 0.9]
    initial_ratios = [0.05, 0.1, 0.2]

    for N in network_sizes:
        networks = create_networks(N)
        for name, G in networks.items():
            nx.write_pajek(G, f"Networks/{name}_{N}.net")
            all_results = []

            plt.figure(figsize=(10, 6))
            for initial_ratio in initial_ratios:
                for mu in mu_values:
                    mc_results = []
                    mmca_results = []
                    for beta in tqdm(beta_values, desc=f"Simulating {name} N={N}, μ={mu}, Ratio={initial_ratio}"):
                        mc_rho = sis_simulation(G, beta, mu, initial_ratio, 1000, 200)
                        mmca_rho = mmca_prediction(G, beta, mu)
                        mc_results.append(mc_rho)
                        mmca_results.append(mmca_rho)
                        all_results.append({
                            "Beta": beta,
                            "Mu": mu,
                            "Initial_Ratio": initial_ratio,
                            "MC_Rho": mc_rho,
                            "MMCA_Rho": mmca_rho
                        })
                    plt.plot(beta_values, mc_results, label=f'MC μ={mu}, Ratio={initial_ratio}')
                    plt.plot(beta_values, mmca_results, linestyle='--', label=f'MMCA μ={mu}, Ratio={initial_ratio}')

            plt.title(f'SIS Model on {name} Network (N={N})')
            plt.xlabel('Beta (Infection Probability)')
            plt.ylabel('Rho (Fraction of Infected Nodes)')
            plt.legend()
            plt.grid(True)
            plt.savefig(f"Plots/Comparison_{name}_N{N}.png")
            plt.close()

            results_df = pd.DataFrame(all_results)
            # Save all results for the current network in one consolidated CSV file
            results_df.to_csv(f"Data/{name}_N{N}_results.csv", index=False)

if __name__ == '__main__':
    run_simulations()


Simulating Erdos_Renyi N=500, μ=0.1, Ratio=0.05:  18%|█▊        | 9/51 [00:03<00:17,  2.43it/s]/var/folders/hv/8r0914b93k5__tgxzywr4jf80000gn/T/ipykernel_37810/3159949729.py:46: RuntimeWarning: overflow encountered in multiply
  new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
/var/folders/hv/8r0914b93k5__tgxzywr4jf80000gn/T/ipykernel_37810/3159949729.py:46: RuntimeWarning: invalid value encountered in matmul
  new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
Simulating Small_World N=500, μ=0.3, Ratio=0.05:  41%|████      | 21/51 [00:14<00:16,  1.78it/s]/var/folders/hv/8r0914b93k5__tgxzywr4jf80000gn/T/ipykernel_37810/3159949729.py:46: RuntimeWarning: overflow encountered in matmul
  new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
Simulating Small_World N=1500, μ=0.9, Ratio=0.2: 100%|██████████| 51/51 [01:29<00:00,  1.76s/it]
